<a href="https://colab.research.google.com/github/prasanth5reddy/D2L/blob/master/Deep%20Learning%20Computation/layers_and_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Libraries

In [1]:
!pip install mxnet

Importing Libraries

In [0]:
from mxnet import nd
from mxnet.gluon import nn

MLP Block

In [3]:
x = nd.random.uniform(shape=(2, 20))

net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(x)


[[ 0.09543004  0.04614332 -0.00286654 -0.07790349 -0.05130243  0.02942037
   0.08696642 -0.0190793  -0.04122177  0.05088576]
 [ 0.0769287   0.03099705  0.00856576 -0.04467199 -0.06926839  0.09132434
   0.06786595 -0.06187842 -0.03436673  0.04234694]]
<NDArray 2x10 @cpu(0)>

A Custom Block

In [0]:
class MLP(nn.Block):
  # Declare a layer with model parameters. Here, we declare two fully
  # connected layers
  def __init__(self, **kwargs):
    # Call the constructor of the MLP parent class Block to perform the
    # necessary initialization. In this way, other function parameters can
    # also be specified when constructing an instance, such as the model
    # parameter, params, described in the following sections
    super(MLP, self).__init__(**kwargs)
    self.hidden = nn.Dense(256, activation='relu') # hidden layer
    self.output = nn.Dense(10)
    
  def forward(self, x):
    # Define the forward computation of the model, that is, how to return the
    # required model output based on the input x
    return self.output(self.hidden(x))

In [5]:
net = MLP()
net.initialize()
net(x)


[[ 0.00362228  0.00633332  0.03201144 -0.01369375  0.10336449 -0.03508018
  -0.00032164 -0.01676023  0.06978628  0.01303309]
 [ 0.03871715  0.02608213  0.03544959 -0.02521311  0.11005433 -0.0143066
  -0.03052466 -0.03852827  0.06321152  0.0038594 ]]
<NDArray 2x10 @cpu(0)>

A Sequential Block

In [0]:
class MySequential(nn.Block):
  def __init__(self, **kwargs):
    super(MySequential, self).__init__(**kwargs)
    
  def add(self, block):
    # Here, block is an instance of a Block subclass, and we assume it has
    # a unique name. We save it in the member variable _children of the
    # Block class, and its type is OrderedDict. When the MySequential
    # instance calls the initialize function, the system automatically
    # initializes all members of _children
    self._children[block.name] = block
    
  def forward(self, x):
    # OrderedDict guarantees that members will be traversed in the order
    # they were added
    for block in self._children.values():
      x = block(x)
    return x

In [7]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(x)


[[ 0.07787765  0.00216401  0.01682201  0.03059879 -0.00702019  0.01668714
   0.04822845  0.00394321 -0.09300036 -0.044943  ]
 [ 0.08891079 -0.00625484 -0.01619132  0.03807178 -0.01451489  0.02006172
   0.0303478   0.02463485 -0.07605445 -0.04389167]]
<NDArray 2x10 @cpu(0)>

Blocks with Code

In [0]:
class FancyMLP(nn.Block):
  def __init__(self, **kwargs):
    super(FancyMLP, self).__init__(**kwargs)
    # Random weight parameters created with the get_constant are not
    # iterated during training (i.e. constant parameters)
    self.rand_weight = self.params.get_constant('rand_weight', nd.random.uniform(shape=(20, 20)))
    self.dense = nn.Dense(20, activation='relu')
    
  def forward(self, x):
    x = self.dense(x)
    # Use the constant parameters created, as well as the relu and dot
    # functions of NDArray
    x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
    # Reuse the fully connected layer. This is equivalent to sharing
    # parameters with two fully connected layers
    x = self.dense(x)
    # Here in Control flow, we need to call asscalar to return the scalar
    # for comparison
    while x.norm().asscalar() > 1:
      x /= 2
    if x.norm().asscalar() < 0.8:
      x *= 10
    return x.sum()

In [9]:
net = FancyMLP()
net.initialize()
net(x)


[25.522684]
<NDArray 1 @cpu(0)>

Nested Blocks

In [0]:
class NestMLP(nn.Block):
  def __init__(self, **kwargs):
    super(NestMLP, self).__init__(**kwargs)
    self.net = nn.Sequential()
    self.net.add(nn.Dense(64, activation='relu'),
                 nn.Dense(32, activation='relu'))
    self.dense = nn.Dense(16, activation='relu')
    
  def forward(self, x):
    return self.dense(self.net(x))

In [11]:
chimera = nn.Sequential()
chimera.add(NestMLP(), nn.Dense(20), FancyMLP())

chimera.initialize()
chimera(x)


[30.518448]
<NDArray 1 @cpu(0)>